In [ ]:
import torchimport torch.nn as nnimport torch.optim as optimimport torchvisionimport torchvision.transforms as transformsfrom torch.utils.data import DataLoaderimport timeimport matplotlib.pyplot as pltimport numpy as np

In [ ]:
# Data loading and preprocessingdef load_cifar10():    # More robust data transforms for AlexNet    transform_train = transforms.Compose([        transforms.Resize((227, 227)),  # Explicitly resize to 227x227 (AlexNet's original input size)        transforms.RandomCrop(227, padding=4),  # Add random cropping for data augmentation        transforms.RandomHorizontalFlip(),  # Add horizontal flip augmentation        transforms.ToTensor(),        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet standard normalization    ])    transform_test = transforms.Compose([        transforms.Resize((227, 227)),  # Consistent resizing for test data        transforms.ToTensor(),        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    ])    try:        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)        trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)        testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)        return trainloader, testloader    except Exception as e:        print(f"Error loading CIFAR-10 dataset: {e}")        raise

In [ ]:
# Training functiondef train_model(model, trainloader, epochs=10, device='cuda'):    try:        model = model.to(device)        model.train()        criterion = nn.CrossEntropyLoss()        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)        train_losses = []        train_accs = []        for epoch in range(epochs):            running_loss = 0.0            correct = 0            total = 0            start_time = time.time()            for i, (inputs, labels) in enumerate(trainloader):                inputs, labels = inputs.to(device), labels.to(device)                optimizer.zero_grad()                outputs = model(inputs)                loss = criterion(outputs, labels)                loss.backward()                optimizer.step()                running_loss += loss.item()                _, predicted = outputs.max(1)                total += labels.size(0)                correct += predicted.eq(labels).sum().item()            epoch_loss = running_loss / len(trainloader)            epoch_acc = 100 * correct / total            epoch_time = time.time() - start_time            train_losses.append(epoch_loss)            train_accs.append(epoch_acc)            print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Time: {epoch_time:.2f}s')            scheduler.step(epoch_loss)        return train_losses, train_accs    except Exception as e:        print(f"Training error: {e}")        raise

In [ ]:
# Evaluation functiondef evaluate_model(model, testloader, device='cuda'):    try:        model = model.to(device)        model.eval()        correct = 0        total = 0        with torch.no_grad():            for inputs, labels in testloader:                inputs, labels = inputs.to(device), labels.to(device)                outputs = model(inputs)                _, predicted = outputs.max(1)                total += labels.size(0)                correct += predicted.eq(labels).sum().item()        accuracy = 100 * correct / total        return accuracy    except Exception as e:        print(f"Evaluation error: {e}")        raise

In [ ]:
# Set epochs and deviceepochs = 5device = 'cuda' if torch.cuda.is_available() else 'cpu'print(f"Using device: {device}")

In [ ]:
trainloader, testloader = load_cifar10()

In [ ]:
# Load and fine-tune the pretrained AlexNet from torchvisionprint("\nFine-tuning pretrained AlexNet...")pretrained_alexnet = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.DEFAULT)# Modify the classifier for CIFAR-10 (10 classes)pretrained_alexnet.classifier = nn.Sequential(    nn.Dropout(p=0.5),    nn.Linear(256 * 6 * 6, 4096),    nn.ReLU(inplace=True),    nn.Dropout(p=0.5),    nn.Linear(4096, 4096),    nn.ReLU(inplace=True),    nn.Linear(4096, 10)  # 10 classes for CIFAR-10)pretrained_losses, pretrained_accs = train_model(pretrained_alexnet, trainloader, epochs=epochs, device=device)pretrained_test_acc = evaluate_model(pretrained_alexnet, testloader, device=device)print(f"Pretrained AlexNet Test Accuracy: {pretrained_test_acc:.2f}%")